## Stock History Year Month Average - ETL Process

In [1]:
# Import Libraries
import time
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo
import os

############################################################

# Delete collection before inserting new data
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
client.Stocks_db.stock_history_average.drop()

############################################################

###############  CBA   ###############
##### Extract #####
# Get the current time in unixtime format
d = datetime.today()
unixtime = time.mktime(d.timetuple())
starting_date = round(time.mktime(datetime.strptime("01/10/1991", "%d/%m/%Y").timetuple()))
current_time = round(unixtime)

user = 'James'
file_directory = 'Desktop\\etl-project\\seperate_collection_etl_processes'
stock = 'CBA'

# Change the download directory
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : f"C:\\Users\\{user}\\{file_directory}"}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chromeOptions)

# Remove the stock history CSV file if it already exists
filename = f'{stock}.AX.csv'
filepath = os.path.join(f"C:\\Users\\{user}\\{file_directory}", filename)
if os.path.exists(filepath):
    os.remove(filepath)

# Get the stock history CSV file
url = f'https://query1.finance.yahoo.com/v7/finance/download/{stock}.AX?period1={starting_date}&period2={current_time}&interval=1d&events=history&includeAdjustedClose=true'
driver.get(url)

time.sleep(5)
driver.quit()
##### Extract #####

##### Transform #####
# Get the stock
df = pd.read_csv(f'{stock}.AX.csv')
# Get the dates and the year of each record
df['Year']= df['Date'].str.slice(start=0,stop=4)
df['Month']= df['Date'].str.slice(start=5,stop=7)
# Remove all the null values
df = df.dropna()
# Get the average 'Open', 'High', 'Low', 'Close', 'Adj Close', and 'Volume' of each month for each year
df_new = df.groupby(['Year','Month']).mean().round(6)
# Round the average 'Volume' to 0 decimal places
df_new['Volume'] = df_new['Volume'].round()
# Round the average 'Open', 'High', 'Low', 'Close', and 'Adj Close' prices to 2 decimal places
df_new[['Open', 'High', 'Low', 'Close', 'Adj Close']] = df_new[['Open', 'High', 'Low', 'Close', 'Adj Close']].round(2)
# Create a new index called 'Year-Month-Average'
df_new = df_new.reset_index()
df_new['Year-Month-Average'] = df_new['Year'] + '-' + df_new['Month']
df_new = df_new.drop(['Year','Month'],axis=1)
df_new = df_new.set_index('Year-Month-Average')
# Transpose the columns
df_new_tran = df_new.copy().T
#Convert to a dictionary
df_dict = df_new_tran.to_dict()
##### Transform #####

##### Load #####
# Load Into Mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

stock_dict = {}

stock_dict[stock] = df_dict

client.Stocks_db.stock_history_average.insert_one(stock_dict)
##### Load #####
###############  CBA   ###############

###############  ANZ   ###############
##### Extract #####
# Get the current time in unixtime format
d = datetime.today()
unixtime = time.mktime(d.timetuple())
starting_date = round(time.mktime(datetime.strptime("01/10/1991", "%d/%m/%Y").timetuple()))
current_time = round(unixtime)

user = 'James'
file_directory = 'Desktop\\etl-project\\seperate_collection_etl_processes'
stock = 'ANZ'

# Remove the stock history CSV file if it already exists
filename = f'{stock}.AX.csv'
filepath = os.path.join(f"C:\\Users\\{user}\\{file_directory}", filename)
if os.path.exists(filepath):
    os.remove(filepath)

# Change the download directory
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : f"C:\\Users\\{user}\\{file_directory}"}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chromeOptions)

# Get the stock history CSV file
url = f'https://query1.finance.yahoo.com/v7/finance/download/{stock}.AX?period1={starting_date}&period2={current_time}&interval=1d&events=history&includeAdjustedClose=true'
driver.get(url)

time.sleep(5)
driver.quit()
##### Extract #####

##### Transform #####
# Get the stock
df = pd.read_csv(f'{stock}.AX.csv')
# Get the dates and the year of each record
df['Year']= df['Date'].str.slice(start=0,stop=4)
df['Month']= df['Date'].str.slice(start=5,stop=7)
# Remove all the null values
df = df.dropna()
# Get the average 'Open', 'High', 'Low', 'Close', 'Adj Close', and 'Volume' of each month for each year
df_new = df.groupby(['Year','Month']).mean().round(6)
# Round the average 'Volume' to 0 decimal places
df_new['Volume'] = df_new['Volume'].round()
# Round the average 'Open', 'High', 'Low', 'Close', and 'Adj Close' prices to 2 decimal places
df_new[['Open', 'High', 'Low', 'Close', 'Adj Close']] = df_new[['Open', 'High', 'Low', 'Close', 'Adj Close']].round(2)
# Create a new index called 'Year-Month-Average'
df_new = df_new.reset_index()
df_new['Year-Month-Average'] = df_new['Year'] + '-' + df_new['Month']
df_new = df_new.drop(['Year','Month'],axis=1)
df_new = df_new.set_index('Year-Month-Average')
# Transpose the columns
df_new_tran = df_new.copy().T
#Convert to a dictionary
df_dict = df_new_tran.to_dict()
##### Transform #####

##### Load #####
# Load Into Mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

stock_dict = {}

stock_dict[stock] = df_dict

client.Stocks_db.stock_history_average.insert_one(stock_dict)
##### Load #####
###############  ANZ   ###############

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\James\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache
[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\James\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache
